In [88]:
#import xml file
import xml.etree.ElementTree as ET
import csv
tree = ET.parse('./Report.xml')
root = tree.getroot()
card_data = []
for card in root.findall(".//card"):
    card_datum = {}
    card_datum["created_stamp"] = card.get("created")
    card_datum["modified_stamp"] = card.get("modified")
    
    entry = card.find("./entry")
    card_datum["traditional"] = entry.find("./headword[@charset='tc']").text

    if entry.find("./cantopron") is None:
        continue
    card_datum["jyutping"] = entry.find("./cantopron").text
    
    if card.find("scoreinfo") is None:
        continue
    card_datum["correct"] = int(card.find("scoreinfo").get("correct"))
    card_datum["incorrect"] = int(card.find("scoreinfo").get("incorrect"))
    card_datum["difficulty"] = int(card.find("scoreinfo").get("difficulty"))
    card_datum["priority"] = card.find("scoreinfo").get("priority")
    card_datum["last_reviewed_stamp"] = int(card.find("scoreinfo").get("lastreviewedtime"))
    card_data.append(card_datum)
    
    

    
    
#convert xml to csv
import pandas as pd
cards_frame = pd.DataFrame(card_data)
for stamp in ["last_reviewed_stamp","created_stamp","modified_stamp"]:
    cards_frame[stamp] = pd.to_datetime(cards_frame[stamp],unit='s')
cards_frame.sort_values("created_stamp", ascending=False,inplace=True)
cards_frame.to_csv('./Report.csv', index = False)


In [89]:
#30 most recently added
cards_frame.sort_values("created_stamp", ascending=False).head(30)

,created_stamp,modified_stamp,traditional,jyutping,correct,incorrect,difficulty,priority,last_reviewed_stamp
808,2021-07-13 20:01:38,2021-08-01 12:23:13,悲,bei1,0,1,90,None,2021-08-01 12:37:50
807,2021-07-13 20:00:59,2021-08-01 12:23:13,悲劇,bei1 kek6,0,1,90,None,2021-08-01 12:36:03
806,2021-07-13 20:00:08,2021-08-01 12:23:13,氹返,tam5 faan2,0,1,90,None,2021-08-01 12:34:56
805,2021-06-29 19:37:14,2021-08-01 12:23:13,蠢,ceon2,3,2,90,veryhigh,2021-07-29 12:22:21
804,2021-06-29 19:29:48,2021-08-01 12:23:13,經過,ging1 gwo3,0,1,90,None,2021-08-01 12:37:00
803,2021-06-29 19:26:24,2021-08-01 12:23:13,打過嚟,daa2 gwo3 lai4,0,1,90,None,2021-08-01 12:34:02
802,2021-06-10 21:03:40,2021-08-01 12:23:13,大話,daai6 waa6,0,1,90,None,2021-08-01 12:34:10
801,2021-06-08 16:43:30,2021-08-01 12:23:13,好人,hou2 jan4,1,0,100,None,2021-08-01 12:37:06
800,2021-06-08 16:43:23,2021-08-01 12:23:13,壞人,waai6 jan4,0,3,90,None,2021-08-01 12:27:05
799,2021-06-08 16:43:10,2021-08-01 12:23:13,冰球,bing1 kau4,0,3,90,None,2021-08-01 12:26:48


In [46]:
#30 most recently reviewed
cards_frame.sort_values("last_reviewed_stamp", ascending=False).head(30)

,created_stamp,modified_stamp,traditional,jyutping,correct,incorrect,difficulty,priority,last_reviewed_stamp
498,2020-10-27 19:08:08,2021-08-01 13:05:20,水龍頭,seoi2 lung4 tau4,0,1,90,None,2021-08-01 18:34:52
502,2020-10-27 22:37:32,2021-08-01 13:06:49,唔記得,m4 gei3 dak1,0,1,90,None,2021-08-01 18:34:46
501,2020-10-27 22:35:23,2021-08-01 13:06:23,收據,sau1 geoi3,0,1,90,None,2021-08-01 18:34:36
507,2020-10-27 22:46:04,2021-08-01 13:08:47,埋單,maai4 daan1,0,1,90,None,2021-08-01 15:53:54
509,2020-10-27 22:48:48,2021-08-01 13:10:04,號碼,hou6 maa5,0,1,90,None,2021-08-01 15:52:31
496,2020-10-27 16:04:35,2021-08-01 13:05:06,間房,gaan3 fong2,0,1,90,None,2021-08-01 15:33:20
494,2020-10-27 15:57:39,2021-08-01 13:04:50,小姐,siu2 ze2,1,0,100,None,2021-08-01 13:10:57
500,2020-10-27 22:33:19,2021-08-01 13:05:36,幫,bong1,1,0,100,high,2021-08-01 13:10:45
789,2021-05-27 18:42:31,2021-08-01 12:23:13,火腿腸,fo2 teoi2 coeng2,1,3,90,low,2021-08-01 12:50:36
808,2021-07-13 20:01:38,2021-08-01 12:23:13,悲,bei1,0,1,90,None,2021-08-01 12:37:50


# Intro to Data Analysis with Pandas & Jupyter, November 2019

Workshop lead: Sam Bail [@spbail](http://twitter.com/spbail)

In [47]:
#30 random words
cards_frame.sample(frac=1).head(30)

,created_stamp,modified_stamp,traditional,jyutping,correct,incorrect,difficulty,priority,last_reviewed_stamp
190,2020-04-22 11:06:41,2021-07-23 16:14:07,司機,si1 gei1,4,11,70,None,2021-02-07 20:08:41
729,2021-04-08 18:56:08,2021-07-23 16:14:07,油,jau4,4,0,100,None,2021-05-24 16:40:03
50,2020-03-21 02:33:59,2021-07-23 16:14:07,灰色,fui1 sik1,23,7,80,None,2021-03-20 17:10:44
387,2020-06-17 13:46:35,2021-07-23 16:14:07,風景,fung1 ging2,6,11,50,None,2021-04-10 02:36:23
78,2020-03-25 00:36:59,2021-07-23 16:14:07,華裔,waa4 jeoi6,14,14,84,None,2021-03-16 17:24:53
235,2020-04-29 13:58:03,2021-07-23 16:14:07,歌手,go1 sau2,5,13,56,None,2021-02-07 20:02:40
332,2020-06-08 02:08:48,2021-07-23 16:14:07,銀行,ngan4 hong4,7,2,78,None,2021-03-24 22:43:18
514,2020-10-28 20:30:21,2021-07-23 16:14:07,冇可能,mou5 ho2 nang4,2,3,90,veryhigh,2021-07-26 12:32:02
714,2021-03-26 02:29:36,2021-07-23 16:14:07,好冇,hou2 mou2,0,1,90,None,2021-03-26 15:08:36
454,2020-08-22 15:46:24,2021-07-23 16:14:07,游水池,jau4 seoi2 ci4,3,1,90,None,2020-12-02 17:51:56


In [48]:
#cards_frame.jyutping.str.contains('tam5')
cards_frame[cards_frame.jyutping.str.contains('tau4')]

,created_stamp,modified_stamp,traditional,jyutping,correct,incorrect,difficulty,priority,last_reviewed_stamp
787,2021-05-27 18:38:30,2021-07-23 16:14:07,骨頭,gwat1 tau4,1,1,90,None,2021-06-08 17:03:47
745,2021-04-26 20:38:33,2021-07-23 16:14:07,頭先,tau4 sin1,1,3,88,None,2021-05-27 19:24:15
738,2021-04-26 20:30:07,2021-07-23 16:14:07,白頭髮,baak6 tau4 faat3,1,2,90,None,2021-05-24 16:45:16
735,2021-04-26 20:26:45,2021-07-23 16:14:07,頭,tau4,2,0,96,None,2021-05-22 15:56:47
712,2021-03-26 02:28:29,2021-07-23 16:14:07,上頭,soeng6 tau4,0,1,92,None,2021-03-26 15:06:43
556,2020-12-24 03:26:52,2021-07-23 16:14:07,剪頭髮,zin2 tau4 faat3,2,3,80,None,2021-05-27 20:11:05
498,2020-10-27 19:08:08,2021-08-01 13:05:20,水龍頭,seoi2 lung4 tau4,0,1,90,None,2021-08-01 18:34:52
461,2020-08-22 15:53:34,2021-07-23 16:14:07,頭髮,tau4 faat3,3,4,70,None,2021-03-16 18:27:25
184,2020-04-15 20:37:54,2021-07-23 16:14:07,蒜頭,syun3 tau4,4,23,70,None,2021-03-16 16:58:52
155,2020-04-10 19:14:50,2021-07-23 16:14:07,石頭,sek6 tau4,6,25,52,None,2021-04-13 21:11:04


# Overview
The goal of this workshop is to give learners an intro to data analysis with Python using Pandas and Jupyter. 
We will first go through the process of loading data from CSV files, inspecting and cleaning the data. As a second step, we will analyse the data and draw some insights about cancer treatment from it. 

The tutorial is structured as follows:

- Intro and background
- Part 0: Quick Jupyter exercise
- Part 1: Loading and inspecting data
- Part 2: Data cleaning
- Part 3: Data analysis
- Part 4: Summary

**Note that this tutorial is only intended as an introduction to some basic concepts of Pandas. It is in no means intended to be comprehensive, and there are a lot of useful functions a beginner needs to know to do in-depth data analysis. We hope that this tutorial sets you up for self-guided learning to master the full range of necessary Pandas tools.**

## How to follow along with the workshop
- You can run every cell in the notebook as we go along using the shortcut Shift+Enter
- You will encounter a few <span style="color:blue">*** DIY exercise ***</span> blocks where you'll get a few minutes to try out what you've just learned
- Feel free to save and download your notebook from Binder at the end since Binder deletes notebooks after 12 hours.

# Intro

## What is Jupyter (and the Jupyter ecosystem...)?
- **IPython** is an **interactive Python shell** (just type "ipython" to start it)
- **Jupyter** is a Python library that provides a **web-based UI** on top of ipython to create notebooks with code and output
- **JupyterLab** provides some additional **features on top of Jupyter**, e.g. a file browser
- **Binder** is a **web-based hub** for containers that contain your Python environment and renders notebooks based on a git repo

## What is Pandas/Matplotlib/Pyplot/Seaborn?

- **Pandas** is a Python library for **data manipulation and analysis**. It offers data structures and operations for manipulating numerical tables and time series.
- **Matplotlib** is a Python **2D plotting library**. Pyplot is a collection of command style functions in matplotlib that make matplotlib work like MATLAB. While we mostly use Seaborn, we sometimes fall back to using Pyplot functions for certain aspects of plotting.
- **Seaborn** is a Python **data visualization** library based on matplotlib. It's kind of like a nicer version of Pyplot.
- You can **use Pandas code in a regular Python script** of course. I'm just combining Jupyter + Pandas in this tutorial because notebooks are a great way to immediately see output!

# Part 0: Quick Jupyter exercise (10 mins)

### Notebooks are basically just interactive ipython terminals, often mixed in with markdown text:
- Each input field you see is called a **cell**
- Cells can be **either code or markdown**
- You can execute any kind of Python code
- **Variables persist** between cells
- The notebook **doesn't care about the order of cells**, just the order of executing it in order to remember variables. However, "run all" executes your cells top to bottom.

### Notebooks have **two modes**: a) editing the cells and b) navigating the notebook (command mode):
- You can **navigate** around the notebook in command mode by clicking cells or using the arrow keys
- Depending on the environment you're using (Jupyter notebook, Jupyter lab, Google Colab...) there will be a different **visual cue** (e.g. a colored line) to indicate the mode a cell is in
- In order to **edit a cell**, you can press **Enter** or double-click it.
- To **execute** the cell content, press Shift+Enter to run the cell
- To get **out of edit mode** and back into navigation mode, press the **Escape key**

### Some helpful keyboard shortcuts:
- The **default type for a cell is code**. In command mode, press *m* to make a cell markdown and *y* to make it code
- Press *a* in command mode to create a new cell *above* the current one
- Press *b* in command mode to create a new cell *below* the current one
- *Tab* autocompletes methods (like in IPython)
- *Shift+Tab* shows you the docstring for the outer function of the line your cursor is in
- Press *dd* in command mode to delete a cell. 
- *Cmd+z* undoes operations in the highlighted cell, *z* undoes cell operations in the notebook (e.g. deleting a cell)

In [49]:
# Examples

print('Hello world!')

import math
math.ceil(4.3)

Hello world!


5

### Cell magic in Jupyter
A "cell magic" is a wrapper function that applies to a line or entire cell in a notebook. Jupyter comes with several built-in "magic" functions.
- A single % before the name indicates a line-specific function
- A double %% applies the function to the entire cell

In [50]:
# Line-specific magic to time the statement execution
import numpy
%time numpy.random.normal(size=20)

CPU times: user 42 µs, sys: 12 µs, total: 54 µs
Wall time: 53.9 µs


array([ 0.0954051 ,  1.19861511, -0.57360776,  1.17832622,  2.27851608,
       -0.35161786,  2.13911645,  0.56749383,  0.28780764, -0.87714966,
       -1.03473761, -0.41527223, -1.03224748,  0.20878888,  0.78398346,
        0.36293489,  0.0998608 , -0.78584546, -0.53789268,  1.43532452])

In [51]:
%%time

# The magic applies to the whole cell and must come first!
numpy.random.normal(size=20)

CPU times: user 50 µs, sys: 8 µs, total: 58 µs
Wall time: 58.9 µs


array([-0.89392214,  0.2211276 ,  0.29613005,  0.443973  , -1.29888381,
       -0.21139753, -0.34049573, -0.8351439 ,  0.73621394,  0.20832439,
       -0.61011551, -1.58925169, -0.75444072,  0.80505293,  1.35795726,
        1.37681619, -0.16201478, -0.84729506,  0.67673297,  0.81069913])

## <span style="color:blue">*** DIY exercise ***</span>
Try out using only your keyboard and shortcuts for these two tasks:
- Create a new *markdown* cell below this one, write a few lines and format them to look like a header and bullets.
- Create a new *code* cell above the first one one, import your favorite Python function, check out the docstring, and execute the code (e.g. `os.getcwd()`, `random.random()`, etc.)

# Part 1: Loading and inspecting the data (20 mins)

Before we can start answering questions about the data we need to do a little bit of exploratory analysis.The first thing we need to do when working with a new dataset is to get an idea of what the data looks like. We start by loading the data into memory. Pandas comes with a built-in `read_csv` function that we can use to read CSV files and load them directly to a pandas `DataFrame` object. 

In [52]:
# We need to import the libraries to start with
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# This command makes charts show inline in a notebook
%matplotlib inline

# Making the figures show up a little larger than default size
plt.rcParams['figure.figsize'] = [10,6]

## What is a dataframe?
* A **dataframe** is a **2-dimensional labeled data structure** with columns of potentially different types. You can think of it like a spreadsheet or SQL table, or a dict of Series objects. It is generally the most commonly used Pandas object. 
* Pandas borrows the concept of DataFrame from the statistical programming language R.
* There are a lot of **different ways to read data** into a dataframe - from lists, dicts, CSVs, databases... In this example, we're loading data from a CSV file!

**Let's take a look at the data to familiarize ourselves with the format and data types. In this example, I'm using some treatment data from the oncology domain, including treatment starts and the drugs patients are getting.**

In [53]:
# Read data from a CSV into a dataframe
# This is the data we're going to be working with!
tx = pd.read_csv('./mock_treatment_starts_2016.csv')

In [54]:
# Just typing the name of the dataframe will print the entire output
# If there are too many rows, Jupyter will print the top few and 
# bottom few rows with a "..." to indicate that there are more rows
tx

,PatientID,TreatmentStart,Drug,Dosage
0,PT1,1/14/16,Cisplatin,200
1,PT20,1/2/16,Cisplatin,140
2,PT2,1/10/16,Cisplatin,180
3,PT3,1/24/16,Cisplatin,140
4,PT4,2/14/16,Cisplatin,200
5,PT19,2/10/16,Cisplatin,180
6,PT5,2/6/16,Cisplatin,190
7,PT6,3/1/16,Cisplatin,180
8,PT7,3/1/16,Cisplatin,210
9,PT8,3/19/16,Cisplatin,180


## Inspecting a dataframe using built-in functions
* Most operations on a dataframe happen by applying a function to it using the "." notation, e.g. `my_dataframe.do_something()`
* Let's look at some simple functions that we can apply to Pandas dataframes

In [55]:
# The info() function prints some basic information about the dataframe
# such as the number of columns and rows
# Let's talk about the # column later!
tx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
PatientID         24 non-null object
TreatmentStart    24 non-null object
Drug              24 non-null object
Dosage            24 non-null int64
dtypes: int64(1), object(3)
memory usage: 896.0+ bytes


In [56]:
# The head(n) function shows the first n rows in a dataframe.
# If no n is specified, it defaults to 5 rows.
tx.head()

,PatientID,TreatmentStart,Drug,Dosage
0,PT1,1/14/16,Cisplatin,200
1,PT20,1/2/16,Cisplatin,140
2,PT2,1/10/16,Cisplatin,180
3,PT3,1/24/16,Cisplatin,140
4,PT4,2/14/16,Cisplatin,200


In [57]:
# You can also use the sample() function to get n random rows in 
# the dataframe
tx.sample(5)

,PatientID,TreatmentStart,Drug,Dosage
15,PT12,5/15/16,Cisplatin,1800
10,PT9,3/27/16,Nivolumab,240
11,PT10,4/7/16,Nivolumab,240
3,PT3,1/24/16,Cisplatin,140
2,PT2,1/10/16,Cisplatin,180


In [58]:
# The describe function shows some basic statistics for numeric columns
# We only have one here (Dosage), so this isn't very interesting
tx.describe()

,Dosage
count,24.000000
mean,264.583333
std,328.963843
min,140.000000
25%,180.000000
50%,195.000000
75%,240.000000
max,1800.000000


## Other ways to inspect a dataframe
* There are other operations you can do on a dataframe that don't follow the function notation
* Let's look at a few examples

In [59]:
# Then len function gives us the number of rows in the dataframe
len(tx)

24

In [60]:
# The shape property gives you the number of rows and columns
tx.shape

(24, 4)

In [61]:
# The dtypes property of a dataframe shows the datatypes of 
# every column in a dataframe.
tx.dtypes

PatientID         object
TreatmentStart    object
Drug              object
Dosage             int64
dtype: object

In [62]:
# The columns attribute of a dataframe contains the column names
# We'll talk about the "Index" later!
tx.columns

Index(['PatientID', 'TreatmentStart', 'Drug', 'Dosage'], dtype='object')

## <span style="color:blue">*** DIY exercise ***</span>
Create a new cell below and print the first ten rows of the "tx" dataframe.

## Accessing columns in a dataframe

<span style="color:blue">**Note: We will be applying `head()` to some results in this tutorial to keep the output short. When working with a real dataset, keep in mind that you might be hiding some relevant records if you always use `head()` or `sample()`!**</span>

**Let's assume you just want to print a specific column or row from your dataframe.** 

In Pandas, you can access a specific column using the following notation which returns a **Series** (not a dataframe).

A series is simply a **vector**, aka a 1-dimensional data structure similar to a list.

In [63]:
# Return the PatientID column as a Series
tx['PatientID'].head()

0     PT1
1    PT20
2     PT2
3     PT3
4     PT4
Name: PatientID, dtype: object

In [64]:
# Check the type to show that this indeed returns a Series object
type(tx['PatientID'])

pandas.core.series.Series

In [65]:
# The alternative notation for accessing a column in a dataframe
# Some people prefer the . notation, others the [] notation.
# Personally, I prefer using [] for visibility and consistency
tx.PatientID.head()

0     PT1
1    PT20
2     PT2
3     PT3
4     PT4
Name: PatientID, dtype: object

In [66]:
# And this is how you access two columns of a dataframe.
# Note that this will return a dataframe again, not a series 
# (because a series has only one column...)
# Also note the double square brackets 
# because you're passing a *list* of columns as an argument
tx[['PatientID', 'Dosage']].head()

,PatientID,Dosage
0,PT1,200
1,PT20,140
2,PT2,180
3,PT3,140
4,PT4,200


In [67]:
# Check the type to confirm that this returns a DataFrame type
type(tx[['PatientID', 'TreatmentStart']])

pandas.core.frame.DataFrame

In [68]:
# This way we can now do some more data exploration, 
# e.g. looking at unique patient IDs using the unique function
# which returns an array of values
tx['PatientID'].unique()

array(['PT1', 'PT20', 'PT2', 'PT3', 'PT4', 'PT19', 'PT5', 'PT6', 'PT7',
       'PT8', 'PT9', 'PT10', 'PT11', 'PT16', 'PT12', 'PT13', 'PT14',
       'PT15', 'PT17', 'PT18'], dtype=object)

## <span style="color:blue">*** DIY exercise ***</span>
Create a new cell below and print the list of unique drugs in the dataframe.

## Accessing rows in a dataframe
In addition to slicing by column, we often want to get the record where a column has a specific value, e.g. a specific Patient_ID here. This can be done using the `.loc` function syntax and a boolean statement:

In [69]:
# Access the record(s) where the value in the PatientID column is PT20
tx.loc[tx['PatientID'] == 'PT20']

,PatientID,TreatmentStart,Drug,Dosage
1,PT20,1/2/16,Cisplatin,140
23,PT20,6/2/16,Nivolumab,240


In [70]:
# This is equivalent to the following shorter notation
# I prefer to always use loc to be more explicit
tx[tx['PatientID'] == 'PT20']

,PatientID,TreatmentStart,Drug,Dosage
1,PT20,1/2/16,Cisplatin,140
23,PT20,6/2/16,Nivolumab,240


In [71]:
# You can also use boolean conditions in the selector
tx.loc[(tx['PatientID'] == 'PT20') & (tx['Drug'] == 'Cisplatin')]

,PatientID,TreatmentStart,Drug,Dosage
1,PT20,1/2/16,Cisplatin,140


## <span style="color:blue">*** DIY exercise ***</span>
Create a new cell below and show all rows where the drug dosage for Cisplatin is less than 180.

## Sorting dataframes
Sorting the output of a dataframe can be helpful for visually inspecting or presenting data! Sorting by one or multiple columns is super easy using the `sort_values` function:

In [72]:
# Sort by earliest treatment start date, i.e. in ascending order (default)
tx.sort_values('TreatmentStart').head()

,PatientID,TreatmentStart,Drug,Dosage
2,PT2,1/10/16,Cisplatin,180
0,PT1,1/14/16,Cisplatin,200
1,PT20,1/2/16,Cisplatin,140
3,PT3,1/24/16,Cisplatin,140
5,PT19,2/10/16,Cisplatin,180


In [73]:
# Sort by latest treatment start, i.e. in descending order
tx.sort_values('TreatmentStart', ascending=False).head()

,PatientID,TreatmentStart,Drug,Dosage
21,PT18,6/3/16,Nivolumab,240
23,PT20,6/2/16,Nivolumab,240
22,PT19,6/2/16,Nivolumab,240
20,PT17,6/17/16,Cisplatin,160
19,PT1,6/17/16,Nivolumab,240


In [74]:
# Finally, you can also sort by a list of columns. If you want to 
# change the ascending/descending orders, pass a **list** of 
# booleans to the `ascending` parameter!
tx.sort_values(['PatientID', 'TreatmentStart']).head()

,PatientID,TreatmentStart,Drug,Dosage
0,PT1,1/14/16,Cisplatin,200
19,PT1,6/17/16,Nivolumab,240
11,PT10,4/7/16,Nivolumab,240
12,PT10,4/7/16,Cisplatin,140
13,PT11,4/17/16,Cisplatin,190


## The `inplace` parameter

**Note: Any operations on a dataframe are *not* permanent, i.e. they only modify the current output, but not the actual dataframe. If you want to preserve the sorting, for example, you have to either assign the output to a new variable, or use the `inplace=True` argument. This will not create any output but actually modify the dataframe.**

In [75]:
# Show the dataframe
tx.head()

,PatientID,TreatmentStart,Drug,Dosage
0,PT1,1/14/16,Cisplatin,200
1,PT20,1/2/16,Cisplatin,140
2,PT2,1/10/16,Cisplatin,180
3,PT3,1/24/16,Cisplatin,140
4,PT4,2/14/16,Cisplatin,200


In [76]:
# Use the inplace keyword to modify the dataframe
# Note that you can also sort by a list of columns
tx.sort_values(['PatientID', 'TreatmentStart'], inplace=True)

In [77]:
# Check out the permanently sorted dataframe
tx.head()

,PatientID,TreatmentStart,Drug,Dosage
0,PT1,1/14/16,Cisplatin,200
19,PT1,6/17/16,Nivolumab,240
11,PT10,4/7/16,Nivolumab,240
12,PT10,4/7/16,Cisplatin,140
13,PT11,4/17/16,Cisplatin,190


## <span style="color:blue">*** DIY exercise ***</span>
Create a new cell below and sort the dataframe by drug (ascending, i.e. alphabetically) and then dosage (descending order, i.e. highest dosage first).

# Part 2: Data cleaning (15 mins)

In [78]:
# Remember the dtypes property?
# The TreatmentStart column should really be a date, right?
tx.dtypes

PatientID         object
TreatmentStart    object
Drug              object
Dosage             int64
dtype: object

## Date conversion

Right away we can see that the date field TreatmentDate is stored as string (object). It might be useful to convert it to **Datetime** objects so that we can perform common date arithmetic on them, like checking if a date came before or after another date, or calculating the number of days between two dates.

In [79]:
# This assigns the datetime version of the TreatmentStart column 
# to a column with the same name
tx['TreatmentStart'] = pd.to_datetime(tx['TreatmentStart'])

In [80]:
# Check the types now - we have a datetime64 type!
tx.dtypes

PatientID                 object
TreatmentStart    datetime64[ns]
Drug                      object
Dosage                     int64
dtype: object

In [81]:
# This is the alternative notation to access a column in a dataframe
tx.TreatmentStart = pd.to_datetime(tx.TreatmentStart)

## Sidebar: Copying dataframes and dropping columns
Sometimes you might want to copy a dataframe, e.g. to do further transformations on it but keep the original.

**Note** that if you assign a dataframe to a new variable, it will reference the same underlying object as the original dataframe. This means that any modification you make to the new dataframe will also be applied to the old one. Use the `copy()` function to make a new copy of the dataframe by value.

In [82]:
txcopy = tx

# Create a new dummy column in the "copy" of our dataframe
txcopy['NewColumn'] = 1

In [83]:
# txcopy now has the new column...
txcopy.head()

,PatientID,TreatmentStart,Drug,Dosage,NewColumn
0,PT1,2016-01-14,Cisplatin,200,1
19,PT1,2016-06-17,Nivolumab,240,1
11,PT10,2016-04-07,Nivolumab,240,1
12,PT10,2016-04-07,Cisplatin,140,1
13,PT11,2016-04-17,Cisplatin,190,1


In [84]:
# ... but so does our original dataframe. This is not really what we want!
tx.head()

,PatientID,TreatmentStart,Drug,Dosage,NewColumn
0,PT1,2016-01-14,Cisplatin,200,1
19,PT1,2016-06-17,Nivolumab,240,1
11,PT10,2016-04-07,Nivolumab,240,1
12,PT10,2016-04-07,Cisplatin,140,1
13,PT11,2016-04-17,Cisplatin,190,1


In [85]:
# Drop the dummy column from the original dataframe
# axis=1 means we're dropping columns, and we need to 
# use inplace=True to make it permanent!
if 'NewColumn' in tx.columns:
    tx.drop('NewColumn', axis=1, inplace=True)

# Then make a *real* copy of the "clean" tx dataframe
txcopy = tx.copy()

# Add the dummy column to tx2 and confirm that the 
# original tx doesn't have it
txcopy['NewColumn'] = 1
tx.head()

,PatientID,TreatmentStart,Drug,Dosage
0,PT1,2016-01-14,Cisplatin,200
19,PT1,2016-06-17,Nivolumab,240
11,PT10,2016-04-07,Nivolumab,240
12,PT10,2016-04-07,Cisplatin,140
13,PT11,2016-04-17,Cisplatin,190


In [86]:
# ... and confirm that the second dataframe does have the column:
tx2.head()

NameError: name 'tx2' is not defined

## <span style="color:blue">*** DIY exercise ***</span>
Create a copy of the tx dataframe called `mytx` and add a new column called  `TreatmentStartDT` that contains the treatment starts as datetime types.

Then print the head() of `mytx.dtypes` to show the datatypes.

# Part 3: Data analysis (30 minutes)
Let's assume we've loaded the treatment related data from a cancer clinic in order to provide them with some analytical insights around the types of drugs they use on their patient population.

## Question 1: Patients treated at the practice

**How many unique patients does the practice treat?**

In [ ]:
# Our data frame contains patient IDs and treatment starts -
# let's check if some patients have multiple treatment starts?
# The unique() function returns the number of unique values 
# in a dataframe column.
print('Number of treatment start records:', len(tx))
print('Number of unique patients who start treatment:', 
      len(tx.PatientID.unique()))

So there are 20 unique patients but we have 24 treatment start records, meaning some patients start multiple treatments in the time that we have data for. This means that if we want to answer the question correctly, we need to make sure to only count unique patients. Let's learn some counting techniques first before coming back to the duplicate issue!

## Question 2: Drugs used at the practice
**What are the drugs used at the practice and how many patients receive those drugs?**

In [ ]:
tx.groupby('Drug')

In [ ]:
# The groupby function works like a groupby in SQL, i.e. it 
# groups the dataframe by the specified column and then lets you 
# apply aggregate functions on the grouped values, 
# e.g. counts, sums, means...
# The count function counts the number of rows with *non-null values* 
# in a column
tx.groupby('Drug').count()

In [ ]:
# Since we are only interested in the number of patients, 
# we can select only the relevant column from the resulting 
# dataframe in the output table

# Note that "PatientID" might not be the best name for this column
# - we can use a rename() function in Pandas to rename it to something 
# like "PatientCount" 
# I'm skipping renaming in this tutorial, but feel free to look it up!
tx.groupby('Drug').count()[['PatientID']]

In [ ]:
# We can also use nunique() which counts the number of *unique* non-null values for each column
# Notice how the numbers are different from the count() result in the TreatmentStart and Dosage columns
tx.groupby('Drug').nunique()

## <span style="color:blue">*** DIY exercise ***</span>
With the techniques you just learned in Question 2, think back to Question 1. Create a new cell below and count how many records each patient has in order to spot those patients that receive more than one treatment.

**Additional point to think about:** Depending on what question we want to answer, counting the number of records might not give us the correct answer. Can you think of different questions a clinic might ask to explain why patients have multiple records? What stands out when using nunique() instead of count()?

### A little bit about indexes in dataframes
Notice that in the above example, the "Drug" column is printed in bold. That's because grouping by it has turned it into the **index** of the resulting dataframe.

The index in a dataframe is the **"row identifier"** - it is generally printed as the column on the left. For example, when we first loaded our data, the index didn't have a name and was just an incrementing integer (scroll up to check!). When you create a groupby object, the index of a resulting dataframe will be the column you group by - in this case, the Drug column became the index.

We frequently **reset** the index in a dataframe for various reasons - in this case, because the index contains data that you want to treat as a column, e.g. for plotting.

In [ ]:
# This is the same groupby we did above. Notice how the "Drug" column is bold
# - it became the index after grouping by it
tx.groupby('Drug').count()[['PatientID']]

In [ ]:
# Reset the index in the grouped dataframe to see what happens:
tx.groupby('Drug').count()[['PatientID']].reset_index()

In [ ]:
# Remember that any operations on the dataframe only modify the output? 
# We didn't *really* group the dataframe or reset the index. 
# The tx dataframe is still the same it was at the beginning.
# We could use inplace=True to make the change permanent.
tx.head()

### Let's plot this! (aka our first Seaborn plot)

In [ ]:
# Let's do the same groupby as above to get the number of patient starts per drug.
# This time, we actually assign the output to a new dataframe `counts` to 
# make the change permanent.
counts = tx.groupby('Drug').count()[['PatientID']].reset_index()

# Let's use a simple bar chart in Seaborn to compare counts for the two drugs
# There are several different ways to do the plotting - this is my preferred style,
# but you might prefer different syntax
fig = sns.barplot(data=counts, x='Drug', y='PatientID')
plt.title('Number of patient starts by drug')
plt.ylabel('Number of patient starts')
plt.xlabel('Drug')
plt.show(fig)

## Question 3: Changes to treatment over time
**Do we see any changes in treatment patterns over time?**

Our data shows treatment starts by date. Let's group these starts by month to see if there are any changes of how many patients start on a given drug over time, e.g. because a new drug got approved.

*Note that the data we're using here is dummy data and pretty artificial - oncology clinics see a much higher volume of patients, and drug uptake is usually slower than shown here.*

In [ ]:
# Let's add a new column that only has the treatment *month* to simplify things
# There are many different ways to do this, we picked a simple one called "astype"
tx['TreatmentStartMonth'] = tx['TreatmentStart'].astype('datetime64[M]')

tx.head()

In [ ]:
# Let's count the number of starts per month per drug to plot it later
# We only want the number of patients, so we filter for that column at the end
drugs_by_month = tx.groupby(['TreatmentStartMonth', 'Drug']).count()[['PatientID']]
drugs_by_month

In [ ]:
# The data already looks interesting... let's plot this 
# Remember to reset_index so we can plot the regular columns
# The "hue" keyword is generally used to distinguish two different 
# categorical variables in plots, e.g. in this case the two different drugs
# NOTE: lineplot() only exists in Seaborn version 0.9 and up
fig = sns.lineplot(data=drugs_by_month.reset_index(), 
                   x='TreatmentStartMonth', 
                   y='PatientID',
                   hue='Drug')
plt.title('Number of patient starts by drug and by month')
plt.ylabel('Number of patient starts')
plt.xlabel('Drug')
plt.show(fig)

## <span style="color:blue">*** DIY exercise ***</span>
Plot `drugs_by_month` as a clustered barplot instead of lineplot. Make sure you're clear about what your x, y, and hue are in this case!

Note that the date labeling on the x axis doesn't look good because Seaborn converts the month back to a datetime. There are several ways to deal with this - can you think of one possible solution that works in this particular case?

## [Optional] Question 4: Dosage and outliers
**Question: What is the average dosage of each drug? Are there any outliers?**

In [ ]:
# An easy first step is to group by the respective drug and use describe()
tx.groupby(['Drug']).describe()

In [ ]:
# This is an example of a more complex way to get aggregates in Pandas
# The agg function takes a dictionary of column:function pairs,
# where "function" can be a built-in function like count, mean, min, etc, 
# or a custom function like a lambda.
tx.groupby(['Drug']).agg({'Dosage': 'mean'})

In [ ]:
# We can also pass a list of functions to a column to get multiple outputs!
tx.groupby(['Drug']).agg({'Dosage': ['count', 'mean', 'std', 'min', 'max']})

In [ ]:
# We can plot this easily in Seaborn - but the outlier squashes our display
fig = sns.boxplot(data=tx, x='Drug', y='Dosage')
plt.show(fig)

In [ ]:
# Use showfliers=False in a boxplot to suppress outliers
fig = sns.boxplot(data=tx, x='Drug', y='Dosage', showfliers=False)
plt.show(fig)

# Part 4: Summary!

We hope this workshop was useful for you. We've only touched on some of the **basic concepts** of Pandas, but we believe this will give you the foundations to keep exploring the data! We covered:

- Basic operations in Jupyter notebooks
- Dataframes and Series in Pandas, and loading data to a dataframe
- Basic data inspection (head, describe, dtypes, accessing columns and rows, sorting)
- Grouping and aggregating (count, nunique)
- Indexing in dataframes and reset_index
- Plotting (bar plots, line plots)

**What we didn't learn:**

This is my (biased) list of very frequent Pandas operations that we didn't cover but you'll likely need for data analysis:
- Joining/merging multiple dataframes
- Filtering and de-duplicating dataframes
- More complex modifications of column values, e.g. filling null values, using lambda functions
- More complex aggregates on grouped dataframes (sum, mean, etc)
- Renaming columns (e.g. renaming an aggregate "PatientID" column to something more meaningful like "PatientCount")

Let me know what you think! samanthapbail@gmail.com / [@spbail](http://twitter.com/spbail)